## Linear Regression



Sklearn based models are slightly finicky to get into a suitable onnx format.
This notebook showcases how to do so using the `hummingbird-ml` python package !

In [23]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"]) #subprocess.check_call is used to execute shell commands in Python.
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "hummingbird-ml"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import os
import torch
import ezkl # A library for zero-knowledge proof (ZKP) operations.
import json
from hummingbird.ml import convert #Provides tools to convert machine learning models into different formats.


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3
reg = LinearRegression().fit(X, y)
reg.score(X, y)

circuit = convert(reg, "torch", X[:1]).model #The convert function from hummingbird.ml is used to convert a scikit-learn model (reg) into a format compatible with PyTorch.






In [24]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [25]:


# export to onnx format
# !!!!!!!!!!!!!!!!! This will flash a warning but it is fine !!!!!!!!!!!!!!!!!!!!!
#Exports the PyTorch model to the ONNX format.
# Input to the model
shape = X.shape[1:]
x = torch.rand(1, *shape, requires_grad=True)
torch_out = circuit(x)
# Export the model
torch.onnx.export(circuit,               # model being run
                  # model input (or a tuple for multiple inputs)
                  x,
                  # where to save the model (can be a file or file-like object)
                  "network.onnx",
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                                'output': {0: 'batch_size'}})

d = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_shapes=[shape],
            input_data=[d],
            output_data=[((o).detach().numpy()).reshape([-1]).tolist() for o in torch_out])

# Serialize data into file:
json.dump(data, open("input.json", 'w'))


In [26]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path)
assert res == True


In [27]:
cal_path = os.path.join("calibration.json")

data_array = (torch.randn(20, *shape).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))

res = await ezkl.calibrate_settings(data_path, model_path, settings_path, "resources")
assert res == True



 <------------- Numerical Fidelity Report (input_scale: 13, param_scale: 13, scale_input_multiplier: 10) ------------->

+-----------------+-----------------+-----------------+-----------------+----------------+------------------+----------------+----------------+--------------------+--------------------+------------------------+
| mean_error      | median_error    | max_error       | min_error       | mean_abs_error | median_abs_error | max_abs_error  | min_abs_error  | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+-----------------+-----------------+-----------------+-----------------+----------------+------------------+----------------+----------------+--------------------+--------------------+------------------------+
| -0.000023841858 | -0.000023841858 | -0.000023841858 | -0.000023841858 | 0.000023841858 | 0.000023841858   | 0.000023841858 | 0.000023841858 | 0.0000000005684342 | -0.0000045569022   | 0.0000045569022        |
+-----------------+---------------

In [28]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [29]:
# srs path
res = await ezkl.get_srs( settings_path)

In [30]:
# now generate the witness file

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [31]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,

    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [32]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,

        "single",
    )

print(res)
assert os.path.isfile(proof_path)

{'instances': [['00a0ed1400000000000000000000000000000000000000000000000000000000']], 'proof': '0x0867887ecfd423bf35b4d4fdddc5094ae86cf761f7cab18245d14c35dff54ece21b18d5a81cf0d5c42974b31229bcc10404736ac2ad8cf845cf4a7c1cd59e8a6178b71b4b4d15b8bbbc3ca2d496c582c4e72eb9f250c4be37898c4bd5bb816c21fa3f5506b503a79cc57d34c87ce0897d85bb8c82c404e3fba36cf00bfef3a9b1b1cc1cdb15c76e0969082828f48fe80cf608cbee9ac6cc4a2f315c86a906731279c292195dede45cd7255f8a649dc9b5a18a9dc38a3cd05a803caad59b610f21911171cceb7b5e7b99f0e36d51f21e0cd7e10d4d05827aecc116ff725735aa126738b7d6a3de2e251cac3d604c7c0007b4ae58784dfd7450d787d747b42a863143b6896c6bf04882e8aee686301a8216cd8301713a33d758cd594aaf9bc64b9169d590471044926bd9ffbf6823e914ba8b666b3644d6ef3468e6660a28109c30c86a43a8e5f9741a96eb6b40750d57c909c19816db4591053645c607e8688a3057bcf7e674dda04c3acd531191fb2bb0e73661a386800dcfb6da4770e5e8bf51c7ba326b3ca2bdc95cb052d59d99f1fa226ab22e7f8960db0b8c8c04e00aa21151a7b154153ed0fc93147d30d2f4c1d6f8b6f6941ac5102978f9496ce4136c51b6923

In [33]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,

    )

assert res == True
print("verified")

verified
